In [ ]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import broadcast

In [ ]:
spark = SparkSession.builder.appName("Optimize").getOrCreate()

In [ ]:
dataframe_logs = spark.read.csv("./data/industrial_monitoring_logs.csv", header=True, inferSchema=True)

In [ ]:
dataframe_logs.explain(True)

In [ ]:
dataframe_repartitioned_8 = dataframe_logs.repartition(8)
dataframe_repartitioned_8.cache()

In [ ]:
dataframe_metadata = spark.read.csv("./data/equipment_metadata.csv", header=True, inferSchema=True)
dataframe_joined = dataframe_repartitioned_8.join(broadcast(dataframe_metadata), dataframe_repartitioned_8.equipment == dataframe_metadata.equipment_type)

In [ ]:
checkpoint_dir = "./checkpoint"
os.makedirs(checkpoint_dir, exist_ok=True)

spark.sparkContext.setCheckpointDir(checkpoint_dir)

print(f"Répertoire de checkpoint configuré : {checkpoint_dir}")

In [ ]:
# Appliquer le checkpoint sur le DataFrame optimisé avec 8 partitions
dataframe_checkpointed = dataframe_repartitioned_8.checkpoint()

print("Checkpoint appliqué sur le DataFrame")
print(f"Nombre de partitions : {dataframe_checkpointed.rdd.getNumPartitions()}")
print(f"Nombre d'enregistrements : {dataframe_checkpointed.count():,}")

In [ ]:
spark.stop()